In [ ]:
import numpy as np
import pandas as pd
artist_conv ={}
with open("artist_conversions.csv", 'r') as f:
    for line in f:
        l = line.split(',')
        a_id = l[0]
        artist_name = l[1]
        artist_name = artist_name.strip()
        artist_conv[a_id]  = artist_name

In [92]:
# Basic algorithm from https://cambridgespark.com/content/tutorials/implementing-your-own-recommender-systems-in-Python/index.html and then changed to suit our needs


import numpy as np
import pandas as pd
header = ['user_id', 'artist_id', 'rating']
df = pd.read_csv('finalUserData.csv', sep=',', names=header)

In [46]:
n_users = df.user_id.unique().shape[0]
n_items = df.artist_id.unique().shape[0]
print ("Number of users = " + str(n_users) + " | Number of movies = " + str(n_items))

Number of users = 615 | Number of movies = 482


In [119]:
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(df, test_size=0.1)

#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [120]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')


In [121]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred


user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [137]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

print ('User-based: ' + str(rmse(user_prediction, test_data_matrix)))
error = 'User-based: ' + str(rmse(user_prediction, test_data_matrix)) + "\n"

User-based: 42.56716483108146


In [131]:
recommendation = []
count = 1
for x in user_prediction: 
    index = x.argsort()[-3:][::-1]
    temp = []
    for y in index:
        temp.append(y)
    main = str(count) + "," + str(artist_conv[str(temp[0])]) + "," + str(artist_conv[str(temp[1])]) + "," + str(artist_conv[str(temp[2])])+"\n"
    print(main)
    recommendation.append(main)
    count = count+1
    

In [138]:
with open('UserBased.txt', 'w') as w:
    for x in recommendation:
        w.write(x)

with open('RMSE.txt', 'a') as w:
    w.write(error)